In [99]:
import numpy as np
import json
import csv
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import optuna
%matplotlib inline

In [100]:
def load_data(df, test_year):
    
    train = df.loc[df["year"] != test_year]
    test = df.loc[df["year"] == test_year]
    
    # 入力データの作成
    y_train = train['prize'].ravel()
    x_train = train.drop(['prize', 'year'], axis=1)
    
    features = x_train.columns
    
    x_train = x_train.values # 学習データ
    
    # テストデータの作成
    y_test = test['prize'].ravel()
    x_test = test.drop(['prize', 'year'], axis=1)
    x_test = x_test.values # テストデータ
    
    return x_train, x_test, y_train, y_test, features

In [101]:
def print_weight_map(weight, features):
    # feature importance
    feature_imp = weight.reshape(features.shape)
    feature_imp_arg = np.argsort(feature_imp)
    f_num = len(feature_imp_arg)
    plt.rcParams["font.family"] = "IPAexGothic"
    plt.figure(figsize=(20,20))
    plt.barh(range(len(feature_imp_arg)), feature_imp[feature_imp_arg], color='b', align='center')
    plt.yticks(range(len(feature_imp_arg)), features[feature_imp_arg])
    plt.show()

In [102]:
def plot_roc_curve(fpr, tpr, auc):
    # ROC曲線をプロット
    plt.plot(fpr, tpr, label='ROC curve (area = %.2f)'%auc)
    plt.legend()
    plt.title('ROC curve')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.grid(True)
    plt.show()

In [103]:
# optunaで最適なC_valを探す

def train(trial):
    
    
    df =  pd.read_pickle('../../data/dataframes/pos_val_df.pkl')
    
    th_prob = 0.5
    C_val = trial.suggest_uniform('C_val', 0.1, 10)
#     l1_ratio = trial.suggest_uniform('l1_ratio', 0, 1)
    
    print(" C_val:{}".format(C_val))
    
    cm_all = np.zeros((2, 2))
    
    # 予測した確率全体を格納
    probs_all = np.array([])
    y_true_all = np.array([])
    
    for y in range(1978, 2020):
        
        # データの生成
        x_train_std, x_test_std, y_train, y_test, features = load_data(df, y)
        
        # 学習
        lr = LogisticRegression(class_weight="balanced", solver="liblinear",  penalty="l1", C=C_val) # ロジスティック回帰モデルのインスタンスを作成
        lr.fit(x_train_std, y_train) # ロジスティック回帰モデルの重みを学習
        
        # 予測(確率)
        probs = lr.predict_proba(x_test_std)
        probs_all = np.hstack((probs_all, probs[:,1]))
        y_true_all = np.hstack((y_true_all, y_test))

        # 混同行列
        y_pred = np.where((probs[:,1] ==  max(probs[:,1])) | (probs[:,1] > th_prob), 1, 0) #確率→0/1
        cm = confusion_matrix(y_true=y_test, y_pred=y_pred)
        cm_all += cm
        
       # 各特徴量の重み係数
        weight = lr.coef_
        
        
#         print_weight_map(weight, features)

    auc = roc_auc_score(y_true_all, probs_all)
    fpr, tpr, thresholds = roc_curve(y_true_all, probs_all)
    
#     print_weight_map(weight, features)
#     plot_roc_curve(fpr, tpr, auc)
    
#     print("len: {0} , {1}".format(len(y_true_all), len(probs_all) ))
#     print("confusion_matrix: ")
#     print(cm_all)
#     print("AUC: ")
#     print(auc)
#     print()
    return -auc

In [104]:
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
@ignore_warnings(category=ConvergenceWarning)
def main():
    np.set_printoptions(precision=3, suppress=True)
    study = optuna.create_study()
    study.optimize(train, n_trials=100)
    print(study.best_trial)

In [105]:
if __name__ == "__main__":
    main()

 C_val:6.585215987426222


[I 2019-08-22 07:44:54,762] Finished trial#0 resulted in value: -0.6370104254719638. Current best value is -0.6370104254719638 with parameters: {'C_val': 6.585215987426222}.


 C_val:9.337508192104684


[I 2019-08-22 07:45:09,158] Finished trial#1 resulted in value: -0.6387010425471964. Current best value is -0.6387010425471964 with parameters: {'C_val': 9.337508192104684}.


 C_val:8.199863663394884


...

 C_val:7.492929887188939


[I 2019-08-22 07:57:56,387] Finished trial#98 resulted in value: -0.6371513102282332. Current best value is -0.6801916032685263 with parameters: {'C_val': 0.4234495565338326}.


 C_val:0.7109574726375503


[I 2019-08-22 07:58:02,330] Finished trial#99 resulted in value: -0.6467314736545506. Current best value is -0.6801916032685263 with parameters: {'C_val': 0.4234495565338326}.


FrozenTrial(number=71, state=<TrialState.COMPLETE: 1>, value=-0.6801916032685263, datetime_start=datetime.datetime(2019, 8, 22, 7, 55, 12, 522334), datetime_complete=datetime.datetime(2019, 8, 22, 7, 55, 15, 385489), params={'C_val': 0.4234495565338326}, distributions={'C_val': UniformDistribution(low=0.1, high=10)}, user_attrs={}, system_attrs={'_number': 71}, intermediate_values={}, params_in_internal_repr={'C_val': 0.4234495565338326}, trial_id=71)


# 結果
C = 0.001

In [74]:
# best value is -0.7222457030149338 with parameters: {'C_val': 0.0013591999321332033}.
# best value is -0.7420400112707805 with parameters: {'C_val': 0.08453576485123458, 'l1_ratio': 0.3231149632136399}.